# Use of static condensation to solve a linear system

When using the spectral element method (SEM) to solve partial differential equations (PDE), it is common for the basis function to have modes with support on boundaries (of elements) and modes with support only inside an element. This structure can be exploited when solving the linear system.

Here, as an exercise for implementing the solver using this technique, we will use it to solve a symmetric linear system that is decomposed as 

$$
A\cdot x  = \left[ 
\begin{matrix} 
A_{bb} & A_{bi} \\
A_{ib} & A_{ii} \\
\end{matrix}\right] \cdot \left\{
\begin{matrix} x_b \\ x_i \\ 
\end{matrix}\right\} = 
\left\{\begin{matrix} f_b \\ f_i \\\end{matrix}\right\} 
$$

where $A_{bb}$, $A_{bi}$, $A_{ib}$ and $A_{ii}$ are submatrices.

The second row can be solved for $x_i$:

$$
x_i = A_{ii}^{-1} f_i - A_{ii}^{-1} A_{ib} x_b
$$

Substituting this on the first row:

$$
\left( A_{bb} - A_{bi}A_{ii}^{-1} A_{ib} \right) x_b = f_b - A_{bi}A_{ii}^{-1} f_i
$$

This can be rewritten as

$$
A'_{bb} \cdot x_b = f'_b
$$

where $A'_{bb} = A_{bb} - A_{bi}A_{ii}^{-1} A_{ib}$ and 
$f'_b = f_b - A_{bi}A_{ii}^{-1} f_i$.

With these transormations, if the linear system is symmetric and positive and definite,  to solve a linear system, we need to use the following steps:
 
 * Compute $A_{ii}^{-1}$ (in reality do a Cholesky decomposition).
 * Compute $A_{bi} A_{ii}^{-1}$
 * Compute the Choleksy factorization of $A'_{bb} = A_{bb} - A_{bi}A_{ii}^{-1} \cdot A_{ib}$
 
To solve the linear system given a RHS (f):

 * Compute $f'_b = f_b - A_{bi}A_{ii}^{-1} f_i$.
 * Using the Cholesky factorization of $A'_{bb}$ compute $x_b$.
 * Now, using the Choleksy factorization of $A_{ii}$ compute $x_i = A_{ii}^{-1} f_i - A_{ii}^{-1} A_{ib} x_b$


## A function to generate random symmetric matrices

In [366]:

function random_matrix(n, symm=true, diag=4.0)
    A = randn(n,n)
    for i = 1:n
        A[i,i] += diag
    end
    
    if symm
        A = (A + A') / 2
    end
    
    return A
end

random_matrix (generic function with 3 methods)

### Creating the matrix

In [367]:
N = 10
nb = 4
ni = N - nb

6

In [368]:
A = [
 3.73295   0.351419  -0.534708   0.301579   0.558047;
  0.351419  6.48719    0.31895    0.198893   0.400289;
 -0.534708  0.31895    4.81739    0.309232  -0.103873;
  0.301579  0.198893   0.309232   5.411     -0.37619 ;
    0.558047  0.400289  -0.103873  -0.37619    3.44621 ]


A = random_matrix(N);


### Obtaining the sub-matrices

In [369]:
Abb = A[1:nb, 1:nb];

In [370]:
Abi = A[1:nb, (nb+1):N];

In [371]:
Aib = A[(nb+1):N, 1:nb];

In [372]:
Aii = A[(nb+1):N, (nb+1):N];

### Loading the appropriate modules

In [373]:
using ArrayViews
using Base.LinAlg.BLAS.gemm!
using Base.LinAlg.BLAS.gemv!
using Base.LinAlg.LAPACK.potrf!
using Base.LinAlg.LAPACK.potrs!


Cholesky decompostion of $A_{ii}$

In [374]:
potrf!('L', Aii);

Computing the matrix $A_{bi}A_{ii}^{-1}$. This is kindy of tricky. Remembering that $(A\cdot B)^T = A^T\cdot B^T$, and that the system is symmetric, if we compute $M = A_{ii}^{-1}\cdot A_{ib}$, then $M^T = \left(A_{ii}^{-1}\cdot A_{ib}\right)^T = A_{bi}\cdot A_{ii}^{-1}$

In [375]:
M = copy(Aib);
potrs!('L', Aii, M);

Now we need to compute $A'_{bb} =  A_{bb} - A_{bi}A_{ii}^{-1} A_{ib} = A_{bb} - M^T\cdot A_{ib}$. BLAS makes it simple:

In [376]:
gemm!('T', 'N', -1.0, M, Aib, 1.0, Abb);

Cholesky decomposition of $A'_{bb}$:

In [377]:
potrf!('L', Abb);

### Solving for a RHS

This is just a test. Let's just create any RHS:

In [378]:
f = float([1:N]);

In [379]:
fbi = copy(f);

In [380]:
fb = view(fbi, 1:nb)
fi = view(fbi, (nb+1):N);

Solving for boundary modes. First we need to correct the RHS: $f'_b = f_b - A_{bi}A_{ii}^{-1} f_i$.

In [381]:
gemv!('T', -1.0, M, fi, 1.0, fb);

Solve the boundary linear system:

In [382]:
potrs!('L', Abb, fb);

Variable fb contains $x_b$. Now solve the equation for $x_i$:

In [383]:
potrs!('L', Aii, fi)
gemv!('N', -1.0, M, fb, 1.0, fi);

In [384]:
x = copy(fbi)

10-element Array{Float64,1}:
  0.905635
  1.58311 
  1.03582 
 -0.54222 
  1.62746 
  2.02464 
  0.478675
  1.14602 
  1.21231 
  2.97395 

Typical solution:

In [385]:
x0 = A\f

10-element Array{Float64,1}:
  0.905635
  1.58311 
  1.03582 
 -0.54222 
  1.62746 
  2.02464 
  0.478675
  1.14602 
  1.21231 
  2.97395 

In [386]:
x0 - x

10-element Array{Float64,1}:
 -1.11022e-16
 -4.44089e-16
 -4.44089e-16
  1.11022e-16
 -2.22045e-16
  4.44089e-16
 -6.66134e-16
 -4.44089e-16
  4.44089e-16
  4.44089e-16